In [15]:
import torch
import torchvision

from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split


import os
import json
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

In [16]:
def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        f.read(16)
        images = np.frombuffer(f.read(), dtype=np.uint8)
        images = images.reshape(-1, 28, 28)
    return images

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        f.read(8)
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

data_dir = "dataset"
train_images_file = os.path.join(data_dir, "train-images.idx3-ubyte")
train_labels_file = os.path.join(data_dir, "train-labels.idx1-ubyte")
test_images_file = os.path.join(data_dir, "t10k-images.idx3-ubyte")
test_labels_file = os.path.join(data_dir, "t10k-labels.idx1-ubyte")

train_images = load_mnist_images(train_images_file)
train_labels = load_mnist_labels(train_labels_file)
test_images = load_mnist_images(test_images_file)
test_labels = load_mnist_labels(test_labels_file)

def save_images(images, labels, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    for i, (image, label) in enumerate(zip(images, labels)):
        class_dir = os.path.join(save_dir, f"class_{label}")
        os.makedirs(class_dir, exist_ok=True)

        img = Image.fromarray(image)
        img.save(os.path.join(class_dir, f"digit_{label}_{i}.jpeg"))

save_images(train_images, train_labels, os.path.join(data_dir, "train"))
save_images(test_images, test_labels, os.path.join(data_dir, "test"))

print("Файлы сохранены в dataset/train и dataset/test")

Файлы сохранены в dataset/train и dataset/test


Пользовательский класс для данных.

In [17]:
class MNISTDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform

        self.len_dataset = 0
        self.data_list = []

        for path_dir, dir_list, file_list in os.walk(path):
            if path_dir == path:
                self.classes = dir_list
                self.class_to_idx = {
                    cls_name: i for i, cls_name in enumerate(self.classes)
                }
                continue

            cls = path_dir.split('/')[-1]

            for name_file in file_list:
                file_path = os.path.join(path_dir, name_file)
                self.data_list.append((file_path, self.class_to_idx[cls]))

            self.len_dataset += len(file_list)

    def __len__(self):
        return self.len_dataset
    
    def __getitem__(self, index):
        file_path, target = self.data_list[index]
        sample = np.array(Image.open(file_path))

        if self.transform is not None:
            sample = self.transform(sample)

        return sample, target

ТЕСТЫ os.walk()

In [18]:
for path, dir_list, file_list in os.walk('dataset/train'):
    print(f'Путь к папке - {path}')
    print(f'    -- кол-во папок {len(dir_list)}')
    print(f'    -- кол-во файлов {len(file_list)}')

Путь к папке - dataset/train
    -- кол-во папок 10
    -- кол-во файлов 0
Путь к папке - dataset/train\class_0
    -- кол-во папок 0
    -- кол-во файлов 5923
Путь к папке - dataset/train\class_1
    -- кол-во папок 0
    -- кол-во файлов 6742
Путь к папке - dataset/train\class_2
    -- кол-во папок 0
    -- кол-во файлов 5958
Путь к папке - dataset/train\class_3
    -- кол-во папок 0
    -- кол-во файлов 6131
Путь к папке - dataset/train\class_4
    -- кол-во папок 0
    -- кол-во файлов 5842
Путь к папке - dataset/train\class_5
    -- кол-во папок 0
    -- кол-во файлов 5421
Путь к папке - dataset/train\class_6
    -- кол-во папок 0
    -- кол-во файлов 5918
Путь к папке - dataset/train\class_7
    -- кол-во папок 0
    -- кол-во файлов 6265
Путь к папке - dataset/train\class_8
    -- кол-во папок 0
    -- кол-во файлов 5851
Путь к папке - dataset/train\class_9
    -- кол-во папок 0
    -- кол-во файлов 5949


In [25]:
train_data = MNISTDataset('dataset/train/')
test_data = MNISTDataset('dataset/test/')
train_data, val_data = random_split(train_data, [0.8, 0.2])

In [ ]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16, shuffle=False)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

(array([[  0,   0,   0,   0,   0,   0,   0,   0,   8,   0,   0,   8,   0,
           0,  15,   0,   0,   5,   5,   0,   0,   0,   0,   6,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,  14,   4,   0,   4,
           0,   0,   1,   0,   4,   0,   2,  17,   8,   0,   4,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   9,   0,   0,  10,   5,
           0,   0,   8,   0,   0,   0,   0,   0,   0,   4,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   8,   0,   8,   1,   0,
          12,  10,   0,   0,   6,   0,   0,   9,  10,   1,   7,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   2,   0,   0,  10,
           0,   0,  57,  57, 172, 245, 255, 243,  96,   0,   0,   0,   0,
           0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   4,   0,   0,   0,   1,
          13,  67, 195, 255, 243, 248, 255, 250, 228, 151,  50,   0,   0,
           0